In [ ]:
from environs import Env
from rich import print as rprint

env = Env()
src = env.path("ENROLL_DIR")
src.exists()

In [ ]:
from foundation.extract_meta import unpack_enroll_data

school_year_meta, enroll_df, levels_df = unpack_enroll_data(enrolment_folder=src)

In [ ]:
school_year_meta.columns

In [ ]:
from foundation.extract_psgc import set_psgc
from foundation.match_psgc_schools import match_psgc_schools

psgc_file = env.path("PSGC_FILE")
psgc_df = set_psgc(f=psgc_file)
meta_psgc = match_psgc_schools(
    psgc_df=psgc_df,
    school_location_df=school_year_meta,
)

In [ ]:
meta_psgc.columns

In [ ]:
import pandas as pd

ADDR_KEY_COLS = [
    "psgc_region_id",
    "psgc_provhuc_id",
    "psgc_muni_id",
    "psgc_brgy_id",
]

rprint("[blue]Building address dimension...[/blue]")
addr_key = meta_psgc[ADDR_KEY_COLS].fillna("").astype(str).agg("|".join, axis=1)

meta_psgc["_addr_hash"] = pd.util.hash_pandas_object(addr_key, index=False)

addresses = (
    meta_psgc[ADDR_KEY_COLS + ["_addr_hash"]]
    .drop_duplicates("_addr_hash")
    .reset_index(drop=True)
)
addresses["address_id"] = addresses.index + 1

addr_df = (
    meta_psgc[["school_id", "school_year", "_addr_hash"]]
    .merge(addresses[["_addr_hash", "address_id"]], on="_addr_hash")
    .drop_duplicates()
)
# SQLite does not have a native int64 type, so we convert to int64
addr_df["_addr_hash"] = addr_df["_addr_hash"].astype(dtype="int64")

In [ ]:
from foundation.extract_geodata import set_coordinates

geo_file = env.path("GEO_FILE")
geo_df = set_coordinates(
    geo_file=geo_file,
    meta_df=meta_psgc.merge(addresses, on="_addr_hash", how="left"),
)

In [ ]:
addr_df.head()

In [ ]:
from sqlite_utils import Database

db = Database(env.path("DB_FILE"), use_counts_table=True)
cols = ["psgc_region_id", "psgc_provhuc_id", "psgc_muni_id", "psgc_brgy_id"]
for col in cols:
    db["geos"].add_foreign_key(col, "psgc", "id")  # type: ignore

db.close()

In [ ]:
from foundation.extract_dataframes import extract_dataframes
from foundation.extract_meta import META_COLS, process_enrollment_folder

In [ ]:
dfs = extract_dataframes()

In [ ]:
psgc_df, enroll_df, geo_df, levels_df, addr_df = dfs

In [ ]:
geo_df

In [ ]:
geo_df.dtypes

In [ ]:
addr_df

In [ ]:
df_long = process_enrollment_folder(src)

In [ ]:
df_long.columns

In [ ]:
(df_long.groupby(["school_year", "school_id"]).size().reset_index(name="num_records"))

In [ ]:
(
    df_long[df_long["school_management"] == "DepEd"]
    .groupby("school_year")["school_id"]
    .nunique()
    .reset_index(name="num_schools")
)

In [ ]:
rprint("[blue]Sorting consolidated enrollment data...[/blue]")
df_sorted = df_long.sort_values(["school_id", "school_year"], ascending=[True, False])

meta = df_sorted[["school_year"] + META_COLS].drop_duplicates(
    subset=["school_id", "school_year"], keep="first"
)

In [ ]:
len(meta)

In [ ]:
(
    meta[meta["school_management"] == "DepEd"]
    .groupby("school_year")["school_id"]
    .nunique()
    .reset_index(name="num_schools")
)